In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os
from pathlib import Path

import numpy as np
from tifffile import imread
import matplotlib.pyplot as plt

from pasnascope import roi, activity

In [ ]:
project_dir = Path(os.getcwd()).parent
img_dir = os.path.join(project_dir, 'data', 'embryos')

measurements = [f for f in sorted(os.listdir(img_dir)) if f.endswith('.csv')]
movies = [f for f in sorted(os.listdir(img_dir)) if f.endswith('ch2.tif')]

## Comparing manual measurements and calculated values

The manually measured values were obtained using ImageJ, and exported as a CSV file.

The calculated values use the `pasnascope` module.

For comparing manual measurements and calculated values, make sure that both have the same sampling interval. 

In [ ]:
def get_vnc_length_from_csv(file_path, interval=20):
    '''Reads csv data as a nparray.

    The csv data contains the manual measurements extracted with ImageJ.
    Column 0 contains indices and column 7 contains Feret's diameter.'''
    data = np.genfromtxt(
        file_path, delimiter=',', skip_header=1, usecols=(0,7))
    indices = data[:, 0]
    indices -= 1
    indices *= interval
    return data

def get_vnc_length_from_tiff(file_path, interval=20, pixel_width=1.62):
    '''Measures VNC length using the `pasnascope` module.'''
    img = imread(file_path)
    img = img[::interval,:,:]
    img_roi = roi.get_roi(img, window=1)
    return activity.measure_VNC(img_roi)*pixel_width

In [ ]:
sampling_interval = 20
PIXEL_WIDTH = 1.62

for csv_file, tiff_file in zip(measurements, movies):
    fig, ax = plt.subplots()
    fig.canvas.header_visible = False
    fig.canvas.resizable = False
    fig.suptitle(tiff_file[:-8])

    manual = get_vnc_length_from_csv(os.path.join(img_dir, csv_file))
    calc = get_vnc_length_from_tiff(os.path.join(img_dir, tiff_file))

    x = manual[:, 0]
    y = manual[:, 1]

    ax.plot(x, y, label='manual')
    ax.plot(x, calc, label='calculated')
    ax.legend()

In [ ]:
sampling_interval = 20
PIXEL_WIDTH = 1.62

errors = []
for csv_file, tiff_file in zip(measurements, movies, strict=True):

    manual = get_vnc_length_from_csv(os.path.join(img_dir, csv_file))
    calc = get_vnc_length_from_tiff(os.path.join(img_dir, tiff_file))

    x = manual[:, 0]
    y = manual[:, 1]

    err = (calc - y)/y
    errors.append((np.average(err), np.std(err)))

fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
fig.suptitle('')

embryo_names = [m[:-8] for m in movies]
avgs, yerr = zip(*errors)

ax.errorbar(embryo_names, avgs, yerr);